In [ ]:
#Generator
mean_A = [3,0]
mean_B = [-2,5]
Ax,Ay = np.random.multivariate_normal(mean_A,[[3.2, 2], [2, 7.0]],500).T
Bx,By = np.random.multivariate_normal(mean_B,[[9.0,4],[4,5]],500).T
points = []
for i in range(0,500):
    # first two are coordinates, third one is the class, last one is the weight of the point
    points.append([Ax[i],Ay[i],1,1./1000])
    points.append([Bx[i],By[i],0,1./1000])

fig = plt.figure()
ax = fig.add_subplot(111)
#plot the data
ax.plot(Ax,Ay,'x')
ax.plot(Bx,By,'o',color='r')
# fix display range
ax.set_ylim([-15,15])
ax.set_xlim([-15,15])

# function to handle the click event
# which is to add a scatter to the point where the user clicks, evaluate the score, get_label and determine if they get wrong
def onclick(event):
    x,y = event.x,event.y
    print 'hi'
    # tag scatter
    ax.plot(x,y,'o',color='y')
    # evaluate score
    score = np.sum([predict(classifier,[x,y])*rates[ind] for ind,classifier in enumerate(candidates)])
    # Let the color be red if it's wrongly predicted
    color = 'g' if pred == ans else 'r'
    ax.text(x,y,str(score),color=color)
    # not sure how it works but it works
    plt.draw_if_interactive()
# deploy onclick listener
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show(block=False)


In [ ]:
#now (Ax,Ay),(Bx,By) are two different sets, start generating random classifier
#prepare classifiers and relevant stuff
def new_classifier():
    w = np.random.rand(2,1) - 0.5
    b = (np.random.rand() - 0.5)*np.random.rand()*10
    return (w,b)

def draw_plane(classifier,col_map = None,val = None):
    w,b = classifier
    w1,w2 = w
    # prepare given x coordinates
    x = np.arange(-15,20,0.1)
    if w2 == 0:
        #vertical line
        y = np.arange(-15,25,0.1)
        x = np.ones(y.size)*(-b)/w1
    else:
        y = -(b + w1*x)/w2
    if col_map:
        return plt.plot(x,y,color=plt.cm.get_cmap(col_map)(val))
    else:
        return plt.plot(x,y,color='g')

def is_correctly_classified(point_with_class,classifier):
    #where classifier is a pair (w,b)
    w,b = classifier
    w1,w2 = w
    x,y,c,score = point_with_class
    if w1*x + w2*y + b >= 0:
        return c == 1
    else:
        return c == 0

# function for normalize weight of the points
def normalize_weight(points):
    weights = [p[3] for p in points]
    sum_weights = np.sum(weights)
    for i in range(0,len(points)):
        points[i][3] = points[i][3]/sum_weights
    return points

# function for raising the weight of the points in target_indices
def raise_weights(target_indices,points):
    for ind in target_indices:
        # ways to increase the weights after each round, this may be decisive to the outcome 
        points[ind][3] *= 15
    return points

# function for predicting the class of a point
def predict(classifier,point):
    x,y = point
    w,b = classifier
    w1,w2 = w
    return 1 if w1*x+w2*y+b > 0 else 0

# function for generating random points
# the coordinates returned should be between (-15,-15) to (15,15) (or in a smaller range, depends on my mood..)
def gen_point():
    x = (np.random.rand() - 0.5) * 15
    y = (np.random.rand() - 0.5) * 15
    return (x,y)

# function to (objectively) determine the label of a given point
# To do this, simply find out which center is the point closer to
def get_label(x,y):
    p = np.array([x,y]).T
    da = np.dot(mean_A,p)
    db = np.dot(mean_B,p)
    return 1 if da > db else 0


In [ ]:
#now to the classification process
#number of boosting rounds
N = 8
candidates = []
# for r in range(0,N):
r = 0
while r < N:
    #normalize the weights of the points
    points = normalize_weight(points)
    #get a bunch of classifiers
    classifiers = [new_classifier() for i in range(0,20)]
    #no mis-classification rate should be higher than 1 as the weights are normalized
    lowest_score = 1
    candidate_index = -1
    mis_classified_points = []
    for index,classifier in enumerate(classifiers):
        score = 0
        for p in points:
            #evaluate the weighted mis-classification rate
            if not is_correctly_classified(p,classifier):
                score += p[3]
        # Draw generated planes with their weights illustrated (or not...)
#         draw_plane(classifier,'Reds',score*10)
        if score < lowest_score:
            lowest_score = score
            candidate_index = index
    # small twist: reject the candidate if the score is way too low,say 0.2
        if lowest_score > 0.5:
            continue
    # get all misclassified points once again for increasing their weights
    target_indices = []
    for ind,p in enumerate(points):
        if not is_correctly_classified(p,classifiers[candidate_index]):
            target_indices.append(ind)
    # record the candidates for this round
    candidates.append(classifiers[candidate_index])
    # increase the weight of the misclassified points
    points = raise_weights(target_indices,points)
    # increment so we don't get an infinite loop
    r += 1

In [ ]:
# output the candidates and their respective rate
# now get the weight of the candidates according to the weight of their
# and get the color map

# normalize the weights first
points = normalize_weight(points)
rates = []
for c in candidates:
    #retrieve the accuracy of first
    rate = np.sum([1 if is_correctly_classified(p,c) else 0 for p in points])
    rates.append(rate/1000.)
    # draw the hyperplane with the given weight
sum_rate = np.sum(rates)
# normalize the accuracy of the candidates again
rates = [r/sum_rate for r in rates]

for ind, c in enumerate(candidates):
    draw_plane(c,'Greens',1 - rates[ind])
    print rates[ind]
# [draw_plane(c,'Greens',weight) for weight in rates]

In [ ]:
# prediction
test_count = 10
# store the error
err = 0

for i in range(0,test_count):
    (x,y) = gen_point()
    #plot the point first
    plt.scatter(x,y,color='y')
    #get the weighted prediction from the candidates
    score = np.sum([predict(classifier,[x,y])*rates[ind] for ind,classifier in enumerate(candidates)])
    #write the label of the prediction
    pred = 0 if score < 0.5 else 1
    # figure out if label is actually correctly classified.
    # Get the answer by other means first
    ans = get_label(x,y)
    # Let the color be red if it's wrongly predicted
    color = 'g' if pred == ans else 'r'
    # write string next to the label
    plt.text(x,y,str(score),color=color)

    # And now I still wish to know the error,
    # i.e. sum of abs(y - s)^2 for y is the label and s is the weighted score of the given point
    err += abs(ans - score)**2

print err